# パッケージのインストール
必要なパッケージをインストールします。

In [ ]:
!pip install -qU torchvision sagemaker

# SageMakerのセッション開始
SageMakerのセッションを開始し、データを保存するS3のプレフィックスと、IAMロールを設定します。

In [ ]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/DEMO-pytorch-mnist"

role_name = "SageMaker-local"
iam = boto3.client("iam")
role = iam.get_role(RoleName=role_name)["Role"]["Arn"]

# 学習用データの取得
学習用データを取得します。)

In [ ]:
from torchvision.datasets import MNIST
from torchvision import transforms

MNIST.mirrors = ["https://sagemaker-sample-files.s3.amazonaws.com/datasets/image/MNIST/"]

MNIST(
    "data",
    download=True,
    transform=transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    ),
)

# S3にデータをアップロード

In [ ]:
inputs = sagemaker_session.upload_data(path="data", bucket=bucket, key_prefix=prefix)
print("input spec (in this case, just an S3 path): {}".format(inputs))

# 学習用のスクリプトの作成
以下のようなスクリプトを作成します。

In [ ]:
!pygmentize mnist.py

# SageMakerで学習を実行する
以下のように学習を実行します。
学習した結果のモデルデータは
`s3://sagemaker-{リージョン名}-{アカウントID}/{トレーニングジョブ名}/model.tar.gz`
に保存されます。

In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point="mnist.py",
    role=role,
    py_version="py38",
    framework_version="1.11.0",
    instance_count=2,
    instance_type="ml.c5.2xlarge",
    hyperparameters={"epochs": 1, "backend": "gloo"},
)

In [ ]:
estimator.fit({"training": inputs})

# エンドポイントを作成して推論を実行する
エンドポイントを作成(つまり、AWSの環境にデプロイしてAWS環境で推論を実行できるようにする)し、推論を実行します。

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

In [ ]:
!ls data/MNIST/raw

In [ ]:
import gzip
import numpy as np
import random
import os

data_dir = "data/MNIST/raw"
with gzip.open(os.path.join(data_dir, "t10k-images-idx3-ubyte.gz"), "rb") as f:
    images = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1, 28, 28).astype(np.float32)

mask = random.sample(range(len(images)), 16)  # randomly select some of the test images
mask = np.array(mask, dtype=np.int)
data = images[mask]

In [ ]:
response = predictor.predict(np.expand_dims(data, axis=1))
print("Raw prediction result:")
print(response)
print()

labeled_predictions = list(zip(range(10), response[0]))
print("Labeled predictions: ")
print(labeled_predictions)
print()

labeled_predictions.sort(key=lambda label_and_prob: 1.0 - label_and_prob[1])
print("Most likely answer: {}".format(labeled_predictions[0]))

# エンドポイントの削除
デプロイしたエンドポイントを削除します。<font color='red'>削除するまで、コンテナの実行費用がかかります。</font>

In [ ]:
sagemaker_session.delete_endpoint(endpoint_name=predictor.endpoint_name)

# Serverless Inferenceを使ったエンドポイントの作成
時々しか実行されず、レスポンス速度が重要ではない場合、Serverless Inferenceを使ってデプロイすると、推論の実行時のみ課金されるので費用が抑えられます。

以下のように、Severless環境を実行するDocker ImageのURIと、学習結果のモデルのS3のURIを指定して、モデルを生成しデプロイします。
`{リージョン名}`と`{アカウントID}`は、適宜変更してください。

In [ ]:
model = sagemaker.pytorch.model.PyTorchModel(
    model_data="s3://sagemaker-{リージョン名}-{アカウントID}/pytorch-training-2022-09-24-07-32-32-301/model.tar.gz",
    entry_point="mnist.py",
    role=role,
    image_uri="763104351884.dkr.ecr.{リージョン名}.amazonaws.com/pytorch-inference:1.12.1-cpu-py38-ubuntu20.04-sagemaker"
)

In [ ]:
from sagemaker.serverless import ServerlessInferenceConfig

serverless_config = ServerlessInferenceConfig(max_concurrency=1)

In [ ]:
predictor = model.deploy(serverless_inference_config=serverless_config)

先程と同様に、推論を実施します。

In [ ]:
response = predictor.predict(np.expand_dims(data, axis=1))
print("Raw prediction result:")
print(response)
print()

labeled_predictions = list(zip(range(10), response[0]))
print("Labeled predictions: ")
print(labeled_predictions)
print()

labeled_predictions.sort(key=lambda label_and_prob: 1.0 - label_and_prob[1])
print("Most likely answer: {}".format(labeled_predictions[0]))

# 備考

本ドキュメントは、[Amazon SageMaker Examples](https://github.com/aws/amazon-sagemaker-examples)のコードを含みます。そのライセンスは、[LICENSE.txt](https://github.com/aws/amazon-sagemaker-examples/blob/main/LICENSE.txt)を参照してください。